##q<sup>2</sup> Tutorial - 4. Elemental abundances

Our hyades_lines.csv file contains rows with atomic data and EWs for C, O, and Ba lines, in addition to the Fe lines we employed before. We can use those data to measure the C, O, and Ba abundances of the stars in our Hyades list.

As usual, let's import q<sup>2</sup> and upload the data. At this point we should use the atmospheric parameters that we derived using q<sup>2</sup>; i.e., not the inputs given in our initial hyades_stars.csv file, but those in the hyades_solution.csv or hyades_solution_odfnew_err.csv files we created in Part 3 of this Tutorial. We will use the latter because we will need to propagate the errors in the parameters to the abundances at some point.

In [1]:
import q2
data = q2.Data('hyades_solution_odfnew_err.csv', 'hyades_lines.csv')

Let's create a star object for hd29310 and load its data. Since we are going to use MOOG to measure the abundances, a model atmosphere is required. We will get one from the odfnew grid (the default).

In [2]:
star = q2.Star('hd29310')
star.get_data_from(data)
star.get_model_atmosphere()

This is all that is needed to calculate the abundances. We just need to tell q<sup>2</sup> which species we are interested in measuring. Our linelist contains not only the C, O, and Ba lines, but also the Fe lines. We can specify which species we want to compute abundances for by passing a species_ids list to q2.abundances.one. The latter is a function that calculates the abundances of elements included in species_ids for a star object (if lines are available). The q2.abundances.one function does not print anything on screen unless its parameter "silent" is set to False (see below). Let's begin with C and Ba, for which CI and BaII lines are available:

In [3]:
species_ids = ['CI', 'BaII']
q2.abundances.one(star, species_ids)

The q2.abundances.one function adds "species" attributes to the star object. For example, star.CI contains the results of the CI abundace calculation. This new attribute is an array, and the line-by-line abundances are in the 'ab' column:

In [4]:
print star.CI['ab']
print star.BaII['ab']

import numpy as np
print "Average C and Ba abundances:"
print "A(C)  = {0:.2f} +/- {1:.2f}".\
      format(np.mean(star.CI['ab']), np.std(star.CI['ab']))
print "A(Ba) = {0:.2f} +/- {1:.2f}".\
      format(np.mean(star.BaII['ab']), np.std(star.BaII['ab']))

[ 8.497  8.575  8.498  8.53 ]
[ 2.307  2.529  2.711]
Average C and Ba abundances:
A(C)  = 8.53 +/- 0.03
A(Ba) = 2.52 +/- 0.17


If we look at the "lines" file again, we will notice that the BaII lines take into account hyperfine structure (HFS). q<sup>2</sup> is able to figure out when HFS is needed and it creates MOOG linelists in the proper format (it uses the "blends" driver instead of "abfind" in these situations).

Now, let's do oxygen:

In [5]:
species_ids = ['OI']
q2.abundances.one(star, species_ids, silent=False)

*** Begin OI:
777 nm oxygen abundances will be NLTE corrected
Wavelength (A) | A(O) LTE | Correction | A(O) NLTE
   7771.9      |   9.001  |    0.177   |  8.824
   7774.2      |   8.995  |    0.172   |  8.823
   7775.4      |   8.934  |    0.150   |  8.784
A(O) LTE  =  8.977 +/- 0.030
A(O) NLTE =  8.810 +/- 0.019
A(OI)  =  8.810 +/- 0.019 (# of lines = 3)
---OI done
All species completed


Oxygen is a special case in q<sup>2</sup>. It is assumed that the oxygen abundances are measured using the 777 nm OI triplet lines. Then, these oxygen abundances are NLTE corrected using the grid by Ramirez et al. (2007). For reference, details on this correction are printed on screen, if requested by the silent parameter (True by default, set to False here to print the information on screen).

All calculations above correspond to absolute elemental abundances. To calculate differential abundances instead, all we need to do is pass a Ref(erence) object to q2.abundances.one. We will use the vestaOct data as reference, creating a ref object and loading its data onto it, in addition to creating a model atmosphere of the same grid (odfnew, the default), for consistency.

Note that we are now calculating abundances for CI only and printing the information on the screen (silent=False). In addition, we are going to calculate the error of this abundance measurement (errors=True). The latter is computed from the line-to-line scatter, quantified by the standard error, added in quadrature with the abundance errors that result from the propagation of errors in the stellar parameters:

In [6]:
ref = q2.Star('vestaOct')
ref.get_data_from(data)
ref.get_model_atmosphere()

species_ids = ['CI']
q2.abundances.one(star, species_ids, Ref=ref, silent=False, errors=True)

*** Begin CI:
A(CI)  =  8.525 +/- 0.032 (# of lines = 4)
[CI/H] =  0.165 +/- 0.040 (# of lines = 4)
-----------------------------
Error propagation for CI:
Line to line scatter:  0.023
Error from Teff:       0.014
Error from logg:       0.015
Error from [Fe/H]:     0.002
Error from vt:         0.001
                      -------
Total abundance error: 0.031
-----------------------------
---CI done
All species completed


As explained before, the species_ids list determines which of all species available in the linelist will be subject to an abundance measurement. If we want to calculate abundances for all species available, then we shold ignore this flag (species_ids=None by default). For example, the command below will calculate the abundances of CI, OI, FeI, FeII, and BaII, i.e., all species included in the linelist, relative to the reference star. It prints important details of the calculations (silent=False) and it measures also the abundance errors (errors=True):

In [7]:
q2.abundances.one(star, Ref=ref, silent=False, errors=True)

"species_ids" not provided
Lines found for the following species: CI,OI,FeI,FeII,BaII

*** Begin CI:
A(CI)  =  8.525 +/- 0.032 (# of lines = 4)
[CI/H] =  0.165 +/- 0.040 (# of lines = 4)
-----------------------------
Error propagation for CI:
Line to line scatter:  0.023
Error from Teff:       0.014
Error from logg:       0.015
Error from [Fe/H]:     0.002
Error from vt:         0.001
                      -------
Total abundance error: 0.031
-----------------------------
---CI done
*** Begin OI:
777 nm oxygen abundances will be NLTE corrected
Wavelength (A) | A(O) LTE | Correction | A(O) NLTE
   7771.9      |   9.001  |    0.177   |  8.824
   7774.2      |   8.995  |    0.172   |  8.823
   7775.4      |   8.934  |    0.150   |  8.784
A(O) LTE  =  8.977 +/- 0.030
A(O) NLTE =  8.810 +/- 0.019
777 nm oxygen abundances will be NLTE corrected (Reference)
Wavelength (A) | A(O) LTE | Correction | A(O) NLTE
   7771.9      |   8.822  |    0.139   |  8.683
   7774.2      |   8.806  |    0.131  

The most practical use of q<sup>2</sup> always corresponds to the case when many stars are handled with just one line of command. If we want to calculate the abundances of C and O for all stars in our csv files using vestaOct as reference and save the results to a file that we will call hyades_co_abundances.csv, we can use the q2.abundances.all function. Note that in this case we do not create star objects, but rather send the entire data object to the function. Thus, here the reference star must be specified by its id, as given in the input stars file:

In [8]:
species_ids = ['CI', 'OI']
q2.abundances.all(data, 'hyades_co_abundances.csv',
                  species_ids, reference='vestaOct')

------------------------------------------------------
Initializing ...
- Date and time: 27-Aug-2015, 16:39:22
- Model atmospheres: odfnew
- Star data: hyades_solution_odfnew_err.csv
- Line list: hyades_lines.csv
- Reference star: vestaOct
------------------------------------------------------

*******
hd28099
*******
Using [Fe/H] =  0.162 for the model atmosphere

CI
--
ABS =  8.499 +/-  0.006 , n = 4
DIF =  0.139 +/-  0.049 , n = 4

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 5052.1670  8.490 -0.009  0.111 -0.028
 5380.3370  8.498 -0.001  0.075 -0.064
 6587.6100  8.508  0.009  0.174  0.035
 7111.4690  8.499  0.000  0.197  0.058

OI
--
ABS =  8.848 +/-  0.018 , n = 3
DIF =  0.164 +/-  0.025 , n = 3

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 7771.9440  8.863  0.015  0.180  0.016
 7774.1610  8.858  0.010  0.183  0.019
 7775.3900  8.822 -0.026  0.129 -0.035

*******
hd29310
*******
Using [Fe/H] =  0.139 for the model

A lot of information is printed on screen for a quick, real-time inspection, but we could also save this as a logfile for future detailed examination.

The output file from this calculation, which we called hyades\_co\_abundances.csv contains both the absolute and relative abundances measured. For example, column CI corresponds to A(C), while [CI] represents [C/H], both measured using CI lines. Columns e\_CI and e\_[CI] contain the 1-sigma line-to-line scatter of the carbon abundances, while n\_CI and n\_[CI] represent the number of lines employed in these calculations.

A more proper error calculation for the abundances must include the propagation of errors from the stellar parameterers. This is achieved by setting the errors flag to True. In the output below, note the lines "ERR = ... ". Those values correspond to the errors discussed here.

In [9]:
q2.abundances.all(data, 'hyades_co_abundances_err.csv',
                  species_ids, 'vestaOct', errors=True)

------------------------------------------------------
Initializing ...
- Date and time: 27-Aug-2015, 16:39:31
- Model atmospheres: odfnew
- Star data: hyades_solution_odfnew_err.csv
- Line list: hyades_lines.csv
- Reference star: vestaOct
------------------------------------------------------

*******
hd28099
*******
Using [Fe/H] =  0.162 for the model atmosphere

CI
--
ABS =  8.499 +/-  0.006 , n = 4
DIF =  0.139 +/-  0.049 , n = 4
ERR = 0.034 (DIF)

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 5052.1670  8.490 -0.009  0.111 -0.028
 5380.3370  8.498 -0.001  0.075 -0.064
 6587.6100  8.508  0.009  0.174  0.035
 7111.4690  8.499  0.000  0.197  0.058

OI
--
ABS =  8.848 +/-  0.018 , n = 3
DIF =  0.164 +/-  0.025 , n = 3
ERR = 0.029 (DIF)

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 7771.9440  8.863  0.015  0.180  0.016
 7774.1610  8.858  0.010  0.183  0.019
 7775.3900  8.822 -0.026  0.129 -0.035

*******
hd29310
*******

In the output file (hyades\_co\_abundances\_err.csv) one can now see another column "err_CI", for example, which contains the abundance error calculations including error propagation from the parameters. Warning: in order for q2 to calculate these errors, the input stellar parameters must also have errors (err\_teff, etc). If you run the command above with a data object that does not include errors in the parameters (or they are all set to 0 or -1) then q<sup>2</sup> will still calculate abundance errors, but assuming that the stellar parameters are perfect!

Finally, the command below calculates the abundances for all elements found in the data object. To achieve this we do not pass a "species_ids" list, but let q<sup>2</sup> figure out all possible species:

In [10]:
q2.abundances.all(data, 'hyades_abundances_err.csv',
                  reference='vestaOct', errors=True)

------------------------------------------------------
Initializing ...
- Date and time: 27-Aug-2015, 16:40:11
- Model atmospheres: odfnew
- Star data: hyades_solution_odfnew_err.csv
- Line list: hyades_lines.csv
- Reference star: vestaOct
------------------------------------------------------
"species_ids" not provided
Lines found for the following species: CI,OI,FeI,FeII,BaII


*******
hd28099
*******
Using [Fe/H] =  0.162 for the model atmosphere

CI
--
ABS =  8.499 +/-  0.006 , n = 4
DIF =  0.139 +/-  0.049 , n = 4
ERR = 0.034 (DIF)

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 5052.1670  8.490 -0.009  0.111 -0.028
 5380.3370  8.498 -0.001  0.075 -0.064
 6587.6100  8.508  0.009  0.174  0.035
 7111.4690  8.499  0.000  0.197  0.058

OI
--
ABS =  8.848 +/-  0.018 , n = 3
DIF =  0.164 +/-  0.025 , n = 3
ERR = 0.029 (DIF)

Wavelength   ABS    RES    DIF    RES 
----------  ----- ------  -----  -----
 7771.9440  8.863  0.015  0.180  0.016
 7774.1610  8.8

The results of this calculation are finally provided in the output file 'hyades\_abundances\_err.csv'. In most cases, we will be interested in columns [CI], err\_[CI], [OI], err\_[OI], etc.

q<sup>2</sup> was written to facilitate the determination of stellar parameters and elemental abundances using MOOG as the synthesizer. It is intended to automate certain steps of this procedure while still allowing human interaction with the data. In Part 5 of this Tutorial we will see how we can use q<sup>2</sup>'s interactive tools to help us improve the precision of our results.